In [1]:
import nibabel as nib
from torch.utils.data import Dataset
from pathlib import Path
import torch
from glob import glob

In [2]:
import matplotlib.pyplot as plt
from torchio.transforms import RandomMotion

/home/rawangq1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ls = list(sorted(Path("/mnt/qdata/share/raecker1/ukbdata_70k/abdominal_MRI/processed/").glob("**/wat.nii.gz")))

In [4]:
ls_small = ls[:10]

In [5]:
import os
from multiprocessing import Pool, current_process


In [11]:

def fileio(filename:str, action:RandomMotion):
    nii = nib.load(filename)
    arr = nii.get_fdata()
    out_arr = action(arr[None])
    print(arr[None].shape)
    new_name = nii.get_filename().replace("processed/","processed/sim/").replace("raecker1","rawangq1").replace("/wat","_wat")
    print(new_name)
    new_nii = nib.Nifti1Image(out_arr, nii.affine)
    nib.save(new_nii, new_name)
def main(filename:str, **kwargs):
    degrees = kwargs["degrees"] if "degrees" in kwargs else 1
    translation = kwargs["translation"] if "translation" in kwargs else 1
    rm = RandomMotion(degrees=degrees, translation=translation)
    fileio(filename, rm)

In [12]:
main(ls[0])

(1, 224, 168, 363)
/mnt/qdata/share/rawangq1/ukbdata_70k/abdominal_MRI/processed/sim/1000029_wat.nii.gz


In [7]:
data_path = "/mnt/qdata/share/raecker1/ukbdata_70k/abdominal_MRI/processed/"
filepaths = list(sorted(Path(data_path).glob("**/wat.nii.gz")))[:10]
os.makedirs(Path(data_path.replace("raecker1","rawangq1"),'sim'), exist_ok=True)
n_process = 4
with Pool(processes=n_process) as P:
    P.map(main, filepaths)
    print(P)
    P.close()
    P.join()
print("\033[93m Fertig \033[0m")


/mnt/qdata/share/rawangq1/ukbdata_70k/abdominal_MRI/processed/sim/1000563/wat.nii.gz


Process ForkPoolWorker-12:
Process ForkPoolWorker-13:
Process ForkPoolWorker-10:
Process ForkPoolWorker-11:


KeyboardInterrupt: 

In [55]:
nii.get_filename().replace("processed/","processed/sim/")

'/mnt/qdata/share/raecker1/ukbdata_70k/abdominal_MRI/processed/sim/1000029/wat.nii.gz'

In [56]:
?nib.save

Signature: nib.save(img: 'FileBasedImage', filename: 'FileSpec', **kwargs) -> 'None'
Docstring:
Save an image to file adapting format to `filename`

Parameters
----------
img : ``SpatialImage``
   image to save
filename : str or os.PathLike
   filename (often implying filenames) to which to save `img`.
\*\*kwargs : keyword arguments
    Keyword arguments to format-specific save

Returns
-------
None
File:      ~/.venv/lib/python3.11/site-packages/nibabel/loadsave.py
Type:      function

In [44]:
import numpy as np

In [45]:
def f(i:np.array,method:RandomMotion):
    o = method(i)

In [48]:
def f1(i:torch.Tensor,method:RandomMotion):
    o = method(i)

In [50]:
%timeit f(nii,rm)

2.11 s ± 45.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%timeit f1(nii_t,rm)

2.23 s ± 74.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
nii.shape

(224, 168, 363)

In [ ]:
class MoCoDataset(Dataset):
    """ MoCoDataset class for loading data from a directory of nifti files.
    Expects data to be cropped into cubes of 64x64x64.
    """
    def __init__(self, sim_path, gt_path):
        self.sim_paths= sorted(Path(sim_path).glob('**/wat.nii*'))
        self.gt_paths = sorted(Path(gt_path).glob('*.nii*'))

    def __len__(self):
        return len(self.sim_paths)

    def __getitem__(self, idx):
        sim = nib.load(self.sim_paths[idx]).get_fdata()
        gt = nib.load(self.gt_paths[idx]).get_fdata()
        return {'GT': torch.from_numpy(gt[None,:,:,:]), 'motion': torch.from_numpy(gt[None,:,:,:])}
    @staticmethod
    def random_split(dataset, ratio:int):
        """
        usage: first build class instance of `MoCoset = MoCoDataset([path1],[path2])`
                then call `MoCoDataset.random_split(MoCoset, ratio=[some int])`
        input: dataset, ratio of split
        return: splited sub-dataset
        """
        generator = torch.Generator().manual_seed(40)
        train_set, val_set = torch.utils.data.random_split(dataset, 
                                                       [len(dataset)//ratio, (len(dataset)-len(dataset)//ratio)],
                                                       generator)
        return train_set, val_set